# Preprocessing and Exploratory analysis of regression dataset downloaded from kaggle

In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import plotly.express as px
from sklearn import metrics

## Preprocessing and One-Hot Encoding

In [76]:
original_train = pd.read_csv('train.csv')
original_train['Week_Number'] = pd.to_datetime(original_train['date']).dt.isocalendar().week
original_train['Month_Number'] = pd.to_datetime(original_train['date']).dt.month
original_train['Year_Number'] = pd.to_datetime(original_train['date']).dt.year

In [78]:
def one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1) #.drop([each], axis=1)
    return df
ohe_train = one_hot(original_train,['Year_Number', 'Week_Number', 'Month_Number', 'product', 'store', 'country'])
ohe_train = ohe_train.drop(['Week_Number', 'Month_Number', 'product', 'store', 'country', 'date', 'row_id'], axis=1)
new_train = ohe_train[ohe_train['Year_Number'] < 2018]
new_test = ohe_train[ohe_train['Year_Number'] == 2018]
new_train = new_train.drop(['Year_Number'],axis=1)
new_test = new_test.drop(['Year_Number'],axis=1)

In [79]:
train_split, validation_split = train_test_split(new_train, test_size=0.2, random_state=123)
type(train_split)

pandas.core.frame.DataFrame

In [80]:
x = train_split.drop('num_sold', axis=1)
y = train_split['num_sold']

regr = linear_model.LinearRegression().fit(x,y)
regr.score(x,y)


0.8092548565470524

In [83]:
regr.coef_
coef_df = pd.DataFrame(zip(x.columns,regr.coef_))
coef_df.columns=['feature_name', 'coeff']

In [86]:
px.bar(coef_df, y='feature_name', x='coeff', orientation='h').show()

In [89]:
validation_x = validation_split.drop(['num_sold'], axis=1)
validation_y = validation_split['num_sold']

In [90]:
regr.predict(validation_x)

array([325.77734375, 174.43359375, 242.88671875, ..., 215.46484375,
       687.1875    , 640.6796875 ])

In [92]:
pred_df = pd.DataFrame(zip(regr.predict(validation_x), validation_y))
pred_df.columns=['predicted_values' , 'true_values']
pred_df

,predicted_values,true_values
0,325.777344,430
1,174.433594,186
2,242.886719,185
3,249.152344,170
4,454.730469,386
...,...,...
3941,670.785156,675
3942,580.925781,463
3943,215.464844,206
3944,687.187500,718


In [94]:
metrics.r2_score(pred_df['true_values'], pred_df['predicted_values'])

0.7992783437071291

In [95]:
px.scatter(x = pred_df['true_values'], y = pred_df['predicted_values']).show()